### Using Gemini

In [32]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

In [34]:
# Load API key from .env file
load_dotenv()
GEMINI_API = os.getenv("GEMINI_API_KEY")

In [35]:
problem_statement="""NYU, an insurance company wants to digitize a range of business processes and provide a complete solution that addresses all aspects of the agent-insurer relationship. Consider yourself as a part of Requirement Analyst team at Retinodes Software Company, and your job is to gather and prioritize the set of requirements. In this new requirement of the project, there are no existing systems that can be analyzed for the development. Requirements have to be gathered, negotiated, validated and prioritized through multiple stakeholders which is a complex process because all stakeholders have different perspectives, requirements and priorities. Therefore, Retinodes want to have a requirements engineering framework that can be used in market-facing projects. To start with, you need to identify the set of stakeholders associated with the system, the domain information about the insurance market, and possible features. The first product NYU wanted you to develop consolidated insurance packages which can compete with the packages provided by other insurance companies. Another product is based on the customer priority, based on the insurance policies available the customer can create his/her own package and send a request for the review. The system has to automatically analyze the package, provide suggestions (if any), and at last give a competing price for the package. To understand the problem domain, existing packages has to be analyzed and the demands and restrictions from the insurance policy and agents have to be understood completely. The requirements and feasibility report generated by you, will further used by the development team for implementation."""

### Identify Stakeholders, End-Users, Functional & Non-functional Requirements

In [36]:
def generate_prompt(problem_statement):
    return f"""
### Role Assignment:
You are a **requirement analyst** responsible for gathering and analyzing system requirements for a **Market-Driven Insurance System**. Your job is to identify stakeholders, end-users, and system requirements based on the given problem statement.

### Task:
Analyze the problem statement and extract the following:
1️⃣ **Stakeholders:** Identify **primary** and **secondary** stakeholders.
2️⃣ **End-users:** List the different categories of end-users.
3️⃣ **Functional Requirements:** Identify system features and functionalities.
4️⃣ **Non-functional Requirements:** Identify system constraints, performance, security, and usability aspects.

### Contextual Information:
- **Primary stakeholders** are directly involved in system development, usage, or decision-making.
- **Secondary stakeholders** are indirectly affected but influence the system.
- **Functional requirements** define the features the system **must perform**.
- **Non-functional requirements** define **how** the system should behave (performance, security, scalability, etc.).

### Instructions:
- **Use Few-Shot Prompting:** Refer to the following example to ensure accurate extraction:
  **Example:**
  🔹 **Primary Stakeholders:** Insurance Company Executives, Business Analysts
  🔹 **Secondary Stakeholders:** Government Regulators, Competing Insurance Firms
  🔹 **End-users:** Insurance Agents, Customers
  🔹 **Functional Requirement:** The system should allow customers to create personalized insurance packages.
  🔹 **Non-functional Requirement:** The system should support **high availability** with 99.9% uptime.

- **Output format:** Provide structured output in a numbered list.

### Problem Statement:
{problem_statement}

### Closing:
If any information is ambiguous or unclear, make logical assumptions and document them explicitly.
"""

In [37]:
# Configure Gemini-Pro model
genai.configure(api_key=GEMINI_API)

# Function to generate response using Gemini-Pro
def run_llm(prompt):
    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        return response.text if response.text else "❌ No response generated."
    except Exception as e:
        return f"❌ ERROR: {e}"

# Function to save output to a text file
def save_output(model_name, output):
    filename = f"{model_name}_output.txt"
    with open(filename, "w", encoding="utf-8") as file:
        file.write(f"Model: {model_name}\n")
        file.write("==================================\n")
        file.write(output + "\n")
    print(f"✅ Output saved to {filename}")

prompt = generate_prompt(problem_statement)

# Run Gemini-Pro and save result
gemini_output = run_llm(prompt)
save_output("gemini-pro", gemini_output)

✅ Output saved to gemini-pro_output.txt


### Gnerate User Stories from Problem Statements

In [38]:
# Function to generate user stories
def generate_user_stories(problem_statement, step1_output):
    prompt = f"""
    You are an expert requirement analyst specializing in user stories following the INVEST framework.

    ## Problem Statement:
    {problem_statement}

    ## Context Specification (Output from Step 1):
    {step1_output}

    ## Guidelines:
    - Generate user stories that are **Independent, Negotiable, Valuable, Estimable, Small, and Testable** (INVEST).
    - Use the format:
        **Front of the Card:**  
        *As a [role], I want to [action] so that [benefit].*  
        **Back of the Card:**  
        - **Success:** 
            (a) [List conditions for success]  
            (b) [List conditions for success]  
        - **Failure - Display Message:** 
            (a) [List error scenarios]
            (b) [List error scenarios]

    ## Few-Shot Examples:

    **Front of the Card:**  
    As a customer, I want to browse and compare insurance packages so that I can choose the best plan for my needs.  

    **Back of the Card:**  
    - **Success:**  
        (a) Display available insurance packages from different providers.  
        (b) Show comparison table with features, pricing, and benefits.  
    - **Failure - Display Message:**  
        (a) "No available insurance packages at the moment, please check later."  
        (b) "System error, please try again."

    **Front of the Card:**  
    As an insurance agent, I want to manage customer policies so that I can assist clients with their insurance needs.  

    **Back of the Card:**  
    - **Success:**  
        (a) View customer policies and their details.  
        (b) Update policy information upon customer request.  
        (c) Generate a new quote based on customer requirements.  
    - **Failure - Display Message:**  
        (a) "Customer policy not found."  
        (b) "Unauthorized access, please log in as an agent."  
        (c) "System error, please try again later."  

    ## Task:
    Based on the problem statement and step1_output files,
    generate 10 user stories following the INVEST format.
    Ensure stories **cover different Stakeholders(Primary & Secondary) and End-users** and **include success and failure cases**.

    Output **ONLY** the user stories in the specified format.
    """
    try:
            model = genai.GenerativeModel("gemini-pro")
            response = model.generate_content(prompt)
            return response.text if response.text else "❌ No response generated."
    except Exception as e:
            return f"❌ ERROR: {e}"

# Function to read content from a file
def read_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        return file.read().strip()

# Define filenames
step1_filename = "gemini-pro_output.txt"
output_filename = "gemini-pro_user_stories_output.txt"

# Read Step 1 Output
step1_output = read_from_file(step1_filename)

# Generate User Stories
print("🚀 Running Gemini-Pro for User Story Generation...")
user_stories_output = generate_user_stories(problem_statement, step1_output)

# Save Output to a .txt File
with open(output_filename, "w", encoding="utf-8") as file:
    file.write(user_stories_output)

print(f"✅ User stories saved successfully in '{output_filename}'!\n")
print("🎉 Gemini-Pro execution completed!")

🚀 Running Gemini-Pro for User Story Generation...
✅ User stories saved successfully in 'gemini-pro_user_stories_output.txt'!

🎉 Gemini-Pro execution completed!


### Chain of Thought for User Story Generation...

In [39]:
# Function to generate user stories using Gemini-Pro with Chain of Thought reasoning
def generate_user_stories_cot(problem_statement, step1_output):
    prompt = f"""
    You are an expert requirement analyst specializing in user stories following the INVEST framework.

    ## Problem Statement:
    {problem_statement}

    ## Context Specification (Output from Step 1):
    {step1_output}

    ## Guidelines:
    - Generate user stories that are **Independent, Negotiable, Valuable, Estimable, Small, and Testable** (INVEST).
    - Use the format:
        **Front of the Card:**  
        *As a [role], I want to [action] so that [benefit].*  
        **Back of the Card:**  
        - **Success:** 
            (a) [List conditions for success]  
            (b) [List conditions for success]  
        - **Failure - Display Message:** 
            (a) [List error scenarios]
            (b) [List error scenarios]

    ## Chain of Thought (CoT) Reasoning:
    Before generating each user story, explicitly follow this reasoning process:
    1️⃣ **What is the user’s goal?**  
    2️⃣ **What feature fulfills this goal?**  
    3️⃣ **What measurable success criteria define a good user story?**  
    4️⃣ **Generate the user story based on this reasoning.**  

    ## Few-Shot Examples (with CoT):

    **Example 1:**  
    **1️⃣ User’s Goal:** Compare different insurance packages to make an informed decision.  
    **2️⃣ Feature:** A comparison page displaying different insurance plans, benefits, and costs.  
    **3️⃣ Success Criteria:** User can view plans, compare key features, and select a package.  

    **Front of the Card:**  
    As a customer, I want to browse and compare insurance packages so that I can choose the best plan for my needs.  

    **Back of the Card:**  
    - **Success:**  
        (a) Display available insurance packages from different providers.  
        (b) Show comparison table with features, pricing, and benefits.  
    - **Failure - Display Message:**  
        (a) "No available insurance packages at the moment, please check later."  
        (b) "System error, please try again."

    **Example 2:**  
    **1️⃣ User’s Goal:** Manage customer policies efficiently.  
    **2️⃣ Feature:** A dashboard where agents can view and edit customer policies.  
    **3️⃣ Success Criteria:** Agents can access, update, and generate policy details.  

    **Front of the Card:**  
    As an insurance agent, I want to manage customer policies so that I can assist clients with their insurance needs.  

    **Back of the Card:**  
    - **Success:**  
        (a) View customer policies and their details.  
        (b) Update policy information upon customer request.  
        (c) Generate a new quote based on customer requirements.  
    - **Failure - Display Message:**  
        (a) "Customer policy not found."  
        (b) "Unauthorized access, please log in as an agent."  
        (c) "System error, please try again later."  

    ## Task:
    Based on the problem statement and identified stakeholders, functional and non-functional requirements, generate **10 user stories** following the INVEST format.  
    **Before each user story, explicitly list the goal, feature, and success criteria using the CoT breakdown.**  
    """

    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        return response.text if response.text else "❌ No response generated."
    except Exception as e:
        return f"❌ ERROR: {e}"

# Function to read content from a file
def read_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        return file.read().strip()

# Define filenames
step1_filename = "gemini-pro_output.txt"
output_filename = "gemini-pro_user_stories_cot_output.txt"

# Read Step 1 Output
step1_output = read_from_file(step1_filename)

# Generate User Stories with CoT
print("🚀 Running Gemini-Pro with Chain of Thought for User Story Generation...")
user_stories_output = generate_user_stories_cot(problem_statement, step1_output)

# Save Output to a .txt File
with open(output_filename, "w", encoding="utf-8") as file:
    file.write(user_stories_output)

print(f"✅ User stories saved successfully in '{output_filename}'!\n")
print("🎉 Gemini-Pro execution completed!")


🚀 Running Gemini-Pro with Chain of Thought for User Story Generation...
✅ User stories saved successfully in 'gemini-pro_user_stories_cot_output.txt'!

🎉 Gemini-Pro execution completed!


###  Identify 3 different EPICs (or collection of user stories) where the conflicts between the requirements occur?

In [40]:
# Function to generate conflicting EPICs
def generate_conflicting_epics(problem_statement, user_stories):
    prompt = f"""
    You are an expert business analyst specializing in requirement conflict resolution.

    ## Problem Statement:
    {problem_statement}

    ## User Stories:
    {user_stories}

    ## Task:
    - Identify **three EPICs** where **conflicts between requirements occur**.
    - Each EPIC should include **two conflicting user stories**.
    - Clearly explain the **conflict**.
    - Propose a **resolution strategy**.

    ## Example EPIC with Conflict:
    
    **EPIC: Policy Customization vs. Standardized Offerings**  
    **Conflicting User Stories:**  
    - As a customer, I want to fully customize my insurance package so that I can get exactly what I need.  
    - As NYU Management, I want to offer only a few well-defined insurance packages so that our operations remain scalable and easy to manage.  
      
    **Conflict:**  
    - Customers want flexibility, but NYU Management prefers structured, pre-defined policies to reduce complexity & risks.  
      
    **Resolution Strategy:**  
    ✅ **Guided Customization:**  
      - Offer pre-configured packages as a starting point.  
      - Allow limited customization within predefined parameters (e.g., adjusting coverage amount, adding riders).  
      - Use AI-driven recommendations to help users choose the best package without making the process overly complex.  

    **Now, generate three EPICs with similar conflict-resolution analysis.**
    """

    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
        return response.text if response.text else "❌ No response generated."
    except Exception as e:
        return f"❌ ERROR: {e}"

# Function to read content from a file
def read_from_file(filename):
    with open(filename, "r", encoding="utf-8") as file:
        return file.read().strip()

# Define filenames
user_stories_filename = "gemini-pro_user_stories_output.txt"
epics_output_filename = "gemini-pro_conflicting_epics.txt"

# Read User Stories Output
user_stories = read_from_file(user_stories_filename)

# Generate Conflicting EPICs
print("🚀 Running Gemini-Pro to identify conflicting EPICs...")
conflicting_epics_output = generate_conflicting_epics(problem_statement, user_stories)

# Save Output to a .txt File
with open(epics_output_filename, "w", encoding="utf-8") as file:
    file.write(conflicting_epics_output)

print(f"✅ Conflicting EPICs saved successfully in '{epics_output_filename}'!\n")
print("🎉 Gemini-Pro execution completed!")


🚀 Running Gemini-Pro to identify conflicting EPICs...
✅ Conflicting EPICs saved successfully in 'gemini-pro_conflicting_epics.txt'!

🎉 Gemini-Pro execution completed!
